In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S')

In [3]:
def load_data(filepath: str):
    'Returns the dataframe'
    try:
        df = pd.read_csv(filepath) # load csv into the env as a pandas dataFrame
        #prints out the number of rows and columns (5630 rows, 20 columns)
        logging.info(f'Data successfully loaded with {df.shape[0]} rows and {df.shape[1]} columns\n')
        return df
    except FileNotFoundError:
        logging.info('File Not Found! Please check filepath and try again')
        raise

In [4]:
df = pd.read_csv('../data/raw/e-commerce.csv')

In [5]:
df.columns.tolist()

['CustomerID',
 'Churn',
 'Tenure',
 'PreferredLoginDevice',
 'CityTier',
 'WarehouseToHome',
 'PreferredPaymentMode',
 'Gender',
 'HourSpendOnApp',
 'NumberOfDeviceRegistered',
 'PreferedOrderCat',
 'SatisfactionScore',
 'MaritalStatus',
 'NumberOfAddress',
 'Complain',
 'OrderAmountHikeFromlastYear',
 'CouponUsed',
 'OrderCount',
 'DaySinceLastOrder',
 'CashbackAmount']

In [31]:
df['WarehouseToHome'].median()

np.float64(14.0)

In [4]:
# ----dataset overview--------
def dataset_overview(df: pd.DataFrame):
    '''Returns the shape of the dataset (i.e. number of rows and columns), 
    alongside a short descriptive summary statistcs of the dataset'''
    logging.info(f'Number of observations : {df.shape[0]}')
    logging.info(f'Number of features : {df.shape[1]}')
    return df.describe(include='all').T

In [5]:
# -------numeric columns-----------
def numeric_columns(df: pd.DataFrame):
    '''Returns numeric columns, together with their minimum and maximum values'''
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    cols = [col for col in numeric_cols[1:]]
    logging.info(f'\nNumber of Numeric columns : {len(cols)} | Examples : {cols[:3]}\n')

    for i,col in enumerate(numeric_cols,1):
        logging.info(f'\n{i}. {col} - Min: {df[col].min()} - Max: {df[col].max()}\n')
    return numeric_cols

In [6]:
# ------------categorical columns---------
def categorical_columns(df: pd.DataFrame):
    '''Returns categorical columns and their respective unique values'''
    categorical_cols = df.select_dtypes(exclude=[np.number]).columns

    cols = [col for col in categorical_cols]
    logging.info(f'\nNumber of Categorical columns : {len(cols)} | Examples : {cols[:3]}\n')

    for i, col in enumerate(categorical_cols,1):
        uniques = df[col].unique()
        logging.info(f'\n{i}. {col} - Unique: {df[col].nunique()} | Examples : {uniques[:3]}\n')
    return categorical_cols

In [7]:
# Numerical Columns Description
# Churn : Target variable (0 = customer stayed, 1 = customer churned/left)
# Tenure : How long the customer has stayed with the company
# CityTier : Classification of the city where the customer lives
# WarehouseToHome : Distance between the warehouse and the customer's home
# HoursSpendOnApp : Average hours the customer spends on the app per day/week
# NumberOfDeviceRegistered : Number of devices registered to a cutomer's account
# SatisfactionScore: A customer's satisfaction rating 
# NumberOfAddress : How many addresses the customer has saved
# Complain : Whether the customer has filed a complaint or not
# OrderAmountHikeFromlastYear : Percentage increase in order compared to last year
# CouponUsed : Number of coupons used by the customer
# OrderCount : Number of orders placed by the customer
# DaysSinceLastOrder : Number of days since the customer's last order
# CashbackAmount : Total cashback the customer has received

In [8]:
# Categorical columns description
# PreferredLoginDevice - The device most often used to log into the app/site
# PreferredPaymentMode - Payment method most often used
# Gender - Sex of the customer (male/female)
# PreferredOrderCat - Most frequent product category ordered
# MaritalStatus - Marital status of the customer

In [9]:
#missing data
def missing_data(df: pd.DataFrame):
    '''Returns the sum of missing data alongside the percentage of 
    missing values with proportion to the length of the dataframe
    '''
    missing = df.isnull().sum()
    missing = missing[missing>0].sort_values(ascending=False)
    missing_pct = missing / len(df) * 100
    logging.info(f'\nMissing Data \n')
    missing_df = pd.DataFrame({
        'missing value' : missing,
        'missing pct' : missing_pct.round(2)
    })
    display(missing_df)
    return missing_df

In [10]:
#------duplicated rows--------
def duplicate(df: pd.DataFrame):
    '''Returns the duplicates found in the dataset'''
    duplicates  =  df[df.duplicated()]
    logging.info(f'\nNumber of duplicates : {len(duplicates)}\n')
    if len(duplicates) == 0:
        logging.info(f'No duplicates found\n')
    else:
        return duplicates

In [11]:
# ---------outlier detection using IQR--------
def check_outlier(df: pd.DataFrame, col: str):
    '''
        Detects outliers in numeric columns using IQR
    '''
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)

    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    return outliers, lower_bound, upper_bound

In [12]:
def outlier_summary(df: pd.DataFrame, numeric_cols: list[str]):
    result = []
    logging.info('Outlier Summary\n')
    for i,col in enumerate(numeric_cols,1):
        outlier, lower, upper = check_outlier(df, col)
        result .append({
            'index': i,
            'columns' : col,
            'outlier' : len(outlier),
            'Lower Range' : lower,
            'Upper Range' : upper
        })
    summary_df = pd.DataFrame(result)
    display(summary_df)
    return summary_df

In [13]:
def one_hot_encode(df: pd.DataFrame, categorical_columns):
    '''One hot encode all categorical columns'''
    return pd.get_dummies(data=df, columns=categorical_columns,dtype=float)

In [14]:
import os
def save_summary(df: pd.DataFrame, name: str):
    os.makedirs('eda_reports',exist_ok=True)
    path = f'eda_reports/{name}.csv'
    df.to_csv(path, index=False)
    logging.info(f'Saved report: {path}')

In [15]:
def run_eda(filepath):
    df = load_data(filepath)
    overview = dataset_overview(df)
    num_cols = numeric_columns(df)
    cat_cols = categorical_columns(df)

    if 'CouponUsed' in df.columns:
        df['CouponUsed'].fillna(0,inplace=True)

    if 'HourSpendOnApp' in df.columns:
        df['HourSpendOnApp'].fillna(df['HourSpendOnApp'].mean(), inplace=True)
    
    if 'WarehouseToHome' in df.columns:
        df = df.query('WarehouseToHome <= 36')

    if 'Tenure' in df.columns:
        df['Tenure'].fillna(df['Tenure'].median(), inplace=True)

    if 'DaySinceLastOrder' in df.columns:
        df['DaySinceLastOrder'].fillna(df['DaySinceLastOrder'].median(), inplace=True)
    
    if 'OrderAmountHikeFromlastYear' in df.columns:
        df['OrderAmountHikeFromlastYear'].fillna(df['OrderAmountHikeFromlastYear'].mean(), inplace=True)

    if 'OrderCount' in df.columns:
        df['OrderCount'].fillna(df['OrderCount'].mean(), inplace=True)

    missing = missing_data(df)
    duplicates = duplicate(df)
    outliers = outlier_summary(df, num_cols)
    df = one_hot_encode(df, cat_cols)
    logging.info(f'EDA completed successfully!')

    save_summary(overview,'overview')
    save_summary(missing, 'missing_data')
    save_summary(outliers, 'outlier_summary')
    if duplicates is not None:
        save_summary(duplicates,'duplicates')
    
    return {
        'data' : df,
        'overview' : overview,
        'num_cols' :num_cols,
        'cat_cols' : cat_cols,
        'missing' : missing,
        'duplicates' : duplicates,
        'outliers' : outliers,
    }

In [16]:
from autoviz.AutoViz_Class import AutoViz_Class
import pkg_resources
import os

def autoviz_report(
        df: pd.DataFrame = None,
        filename: str = None,
        target: str = None,
        output_dir: str = 'autoviz_reports',
        dep_var: str = None
):
    '''
        Generates a comprehensive Autoviz EDA report from either a dataframe or a file.
        Saves HTML plots in the specified output directory. 
    '''
    os.makedirs(output_dir,exist_ok=True)
    logging.info(f'Running AutoViz eda....Output directory : {output_dir}')

    AV = AutoViz_Class()
    # ---- if a dataframe is passed---
    if df is not None:
        logging.info(f'Using in-memory dataframe with {df.shape[0]} rows and {df.shape[1]} columns')
        dft = AV.AutoViz(
            filename='',
            depVar=target,
            dfte=df,
            verbose=2,
            chart_format='html',
            save_plot_dir=output_dir
        )

    # if a filename is passed instead of a dataframe
    elif filename:
        dft = AV.AutoViz(
            filename='../data/e-commerce.csv',
            depVar=target,
            dfte=None,
            verbose=2,
            chart_format='html',
            save_plot_dir=output_dir
        )

    else:
        raise ValueError('You must either provide a dataframe or a filepath!')
    
    logging.info(f'Autoviz EDA completed. Reports saved successfully')
    return dft

Imported v0.1.905. Please call AutoViz in this sequence:
    AV = AutoViz_Class()
    %matplotlib inline
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)


In [17]:
if __name__ == '__main__':
    results = run_eda('../data/e-commerce.csv')
    df = results['data']

    # run autoviz report
    autoviz_report(df=df, target='Churn', output_dir='autoviz_reports')

14:48:07 - INFO - Data successfully loaded with 5630 rows and 20 columns

14:48:07 - INFO - Number of observations : 5630
14:48:07 - INFO - Number of features : 20
14:48:07 - INFO - 
Number of Numeric columns : 14 | Examples : ['Churn', 'Tenure', 'CityTier']

14:48:07 - INFO - 
1. CustomerID - Min: 50001 - Max: 55630

14:48:07 - INFO - 
2. Churn - Min: 0 - Max: 1

14:48:07 - INFO - 
3. Tenure - Min: 0.0 - Max: 61.0

14:48:07 - INFO - 
4. CityTier - Min: 1 - Max: 3

14:48:07 - INFO - 
5. WarehouseToHome - Min: 5.0 - Max: 127.0

14:48:07 - INFO - 
6. HourSpendOnApp - Min: 0.0 - Max: 5.0

14:48:07 - INFO - 
7. NumberOfDeviceRegistered - Min: 1 - Max: 6

14:48:07 - INFO - 
8. SatisfactionScore - Min: 1 - Max: 5

14:48:07 - INFO - 
9. NumberOfAddress - Min: 1 - Max: 22

14:48:07 - INFO - 
10. Complain - Min: 0 - Max: 1

14:48:07 - INFO - 
11. OrderAmountHikeFromlastYear - Min: 11.0 - Max: 26.0

14:48:07 - INFO - 
12. CouponUsed - Min: 0.0 - Max: 16.0

14:48:07 - INFO - 
13. OrderCount - Min

,missing value,missing pct


14:48:07 - INFO - 
Number of duplicates : 0

14:48:07 - INFO - No duplicates found

14:48:07 - INFO - Outlier Summary



,index,columns,outlier,Lower Range,Upper Range
0,1,CustomerID,0,47261.000000,58501.000000
1,2,Churn,864,0.000000,0.000000
2,3,Tenure,4,-16.500000,35.500000
3,4,CityTier,0,-2.000000,6.000000
4,5,WarehouseToHome,0,-7.500000,36.500000
5,6,HourSpendOnApp,6,0.500000,4.500000
6,7,NumberOfDeviceRegistered,374,1.500000,5.500000
7,8,SatisfactionScore,0,-1.000000,7.000000
8,9,NumberOfAddress,4,-4.000000,12.000000
9,10,Complain,0,-1.500000,2.500000


14:48:07 - INFO - EDA completed successfully!
14:48:07 - INFO - Saved report: eda_reports/overview.csv
14:48:07 - INFO - Saved report: eda_reports/missing_data.csv
14:48:07 - INFO - Saved report: eda_reports/outlier_summary.csv
14:48:07 - INFO - Running AutoViz eda....Output directory : autoviz_reports
14:48:07 - INFO - Using in-memory dataframe with 5377 rows and 36 columns


...

Saving heatmaps in HTML format


Saving cat_var_plots in HTML format
                                               

14:49:05 - WARNING - W-1005 (FIXED_SIZING_MODE): 'fixed' sizing mode requires width and height to be set: Column(id='445ae7da-9265-4f93-b55d-6ea3abb8a1bb', ...)


14:49:05 - INFO - Autoviz EDA completed. Reports saved successfully


Time to run AutoViz (in seconds) = 58


In [19]:
df.to_csv('../data/Preprocessed_e-commerce.csv',index=False)